In [ ]:
#import sqlite3
#import pandas as pd
#con = sqlite3.connect('basketball.sqlite')
#cur = con.cursor()
#rs = cur.execute("SELECT * FROM Player_Salary")
#df_player_salary = pd.DataFrame(rs.fetchall())
#con.close()


In [1]:
!pip install pycodestyle pycodestyle_magic

You should consider upgrading via the 'c:\users\nick\documents\github\data-science-capstone-two\nba_player_salary\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
%%load_ext pycodestyle_magic

UsageError: Cell magic `%%load_ext` not found (But line magic `%load_ext` exists, did you mean that instead?).


In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///basketball.sqlite')
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Player_Salary")
    df_player_salary = pd.DataFrame(rs.fetchall())
    df_player_salary.columns = rs.keys()


In [2]:
print(df_player_salary.head())

  slugSeason       nameTeam         namePlayer    statusPlayer  isFinalSeason  \
0    2020-21  Atlanta Hawks  Bogdan Bogdanovic  current roster              0   
1    2021-22  Atlanta Hawks  Bogdan Bogdanovic  current roster              0   
2    2022-23  Atlanta Hawks  Bogdan Bogdanovic  current roster              0   
3    2023-24  Atlanta Hawks  Bogdan Bogdanovic  current roster              1   
4    2020-21  Atlanta Hawks    Brandon Goodwin  current roster              0   

   isWaived  isOnRoster  isNonGuaranteed  isTeamOption  isPlayerOption  \
0         0           1                0             0               0   
1         0           1                0             0               0   
2         0           1                0             0               0   
3         0           1                0             0               1   
4         0           1                0             0               0   

  typeContractDetail       value  
0         Guaranteed  18000000.0 